In [21]:
import pymysql
import pandas as pd

In [22]:
# initialize conenction given user and password created in sql file with database tweets
connection = pymysql.connect(host='localhost',
                             user='tweetuser',
                             password='password',
                             db='Tweets')


In [27]:
# testing out code
cursor = connection.cursor()
sql = "SELECT * FROM Follows;" # automatically assumes count?
cursor.execute(sql)
result = cursor.fetchall() # fetching it gets the real data

# Convert the result to a DataFrame
df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

df.head()

,user_id,follows_id
0,0,70
1,0,703
2,0,4073
3,0,7124
4,0,9319
